In [1]:
# %load_ext autoreload
# %autoreload 2

import numpy as np
import pandas as pd
import tensorflow as tf
from computer_vision.scripts.models import CNN2
from computer_vision.scripts.utils import acc_at_k, tf_init, output_file
import matplotlib.pyplot as plt

config = tf_init()

%matplotlib inline

/afs/csail.mit.edu/u/n/nhunt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [22]:
fnames = ! ls models/
fnames = [fname for fname in fnames if 'log' not in fname]
fnames

['inception_aug_LR_contrast_.75_hue_.15_rot_4_l2_0.0001_p_0.8_na_2_nb_2_nc_2_bn_before_True_lr_0.001_e700_bs64',
 'inception_aug_LR_contrast_.75_hue_.15_rot_4_l2_0.0001_p_0.8_na_3_nb_4_nc_2_bn_before_True_lr_0.001_e700_bs64',
 'inception_aug_LR_contrast_.75_hue_.15_rot_4_l2_0.0001_p_0.8_na_3_nb_5_nc_3_bn_before_True_lr_0.001_e700_bs64']

In [23]:
dev_fnames = ['/afs/csail.mit.edu/u/n/nhunt/github/computer_vision/miniplaces/data/val_0.tfrecord']
test_fnames = ['/afs/csail.mit.edu/u/n/nhunt/github/computer_vision/miniplaces/data/test_0.tfrecord']

In [ ]:
# can't actually do ensemble predictions on val because we don't know how to sort the predictions into the right order

all_preds = []
all_labels = []

split = 'val'

for fname in fnames:
    try:
        cnn = CNN2(models_dir='models/', model_name=fname)
        if split == 'test':
            preds, labels = cnn._batch([cnn.predict['default'], cnn.labels_p['default']], inputs={'default': test_fnames}, dataset=True)
            labels = np.concatenate(labels)
            preds = np.concatenate(preds)

            idx = np.argsort(labels)
            preds = preds[idx]
            all_preds.append(preds)
        elif split == 'val':
            preds, labels = cnn._batch([cnn.predict['default'], cnn.labels_p['default']], inputs={'default': dev_fnames}, dataset=True)
            labels = np.concatenate(labels)
            preds = np.concatenate(preds)
            
            all_preds.append(preds)
            all_labels.append(labels)
            
            preds = pd.DataFrame(preds)
            print(f"{acc_at_k(1, preds, labels)}, {acc_at_k(5, preds, labels)}, {fname}")
    except:
        print(f"Error with {fname}")

Loading graph from: models//inception_aug_LR_contrast_.75_hue_.15_rot_4_l2_0.0001_p_0.8_na_2_nb_2_nc_2_bn_before_True_lr_0.001_e700_bs64/model.ckpt.meta.
0.2686, 0.5749, inception_aug_LR_contrast_.75_hue_.15_rot_4_l2_0.0001_p_0.8_na_2_nb_2_nc_2_bn_before_True_lr_0.001_e700_bs64
Loading graph from: models//inception_aug_LR_contrast_.75_hue_.15_rot_4_l2_0.0001_p_0.8_na_3_nb_4_nc_2_bn_before_True_lr_0.001_e700_bs64/model.ckpt.meta.
0.3366, 0.6453, inception_aug_LR_contrast_.75_hue_.15_rot_4_l2_0.0001_p_0.8_na_3_nb_4_nc_2_bn_before_True_lr_0.001_e700_bs64


In [27]:
print(f"{acc_at_k(1, preds, labels)}, {acc_at_k(5, preds, labels)}, {fname}")

0.3046, 0.6132, inception_aug_LR_contrast_.75_hue_.15_rot_4_l2_0.0001_p_0.8_na_3_nb_5_nc_3_bn_before_True_lr_0.001_e700_bs64


In [29]:
z = [pr.reshape(*pr.shape, 1) for pr in all_preds]
z = np.concatenate(z, axis=2).mean(axis=2)
z = pd.DataFrame(z)

In [ ]:
print(f"{acc_at_k(1, z, labels)}, {acc_at_k(5, z, labels)}")

In [35]:
cnn = CNN2(models_dir='models/', model_name='inception_aug_LR_contrast_.75_hue_.15_rot_4_l2_0.0001_p_0.8_na_3_nb_4_nc_2_bn_before_True_lr_0.001_e700_bs64')

Loading graph from: models//inception_aug_LR_contrast_.75_hue_.15_rot_4_l2_0.0001_p_0.8_na_3_nb_4_nc_2_bn_before_True_lr_0.001_e700_bs64/model.ckpt.meta.


In [19]:
preds, labels = cnn._batch([cnn.predict['default'], cnn.labels_p['default']], inputs={'default': dev_fnames}, dataset=True)
labels = np.concatenate(labels)
preds = pd.DataFrame(np.concatenate(preds))
print(f"{fname}, {acc_at_k(1, preds, labels)}, {acc_at_k(5, preds, labels)}")

inception_aug_LR_contrast_.75_hue_.15_rot_4_l2_0.0001_p_0.8_na_3_nb_5_nc_3_bn_before_True_lr_0.001_e700_bs64, 0.2945, 0.5944


In [36]:
preds, labels = cnn._batch([cnn.predict['default'], cnn.labels_p['default']], inputs={'default': test_fnames}, dataset=True)
labels = np.concatenate(labels)
preds = np.concatenate(preds)

idx = np.argsort(labels)
preds = preds[idx]
p.append(preds)

In [44]:
output_file(z)

In [27]:
all_preds = []
all_labels = []
cnn.sess.run(cnn.data_init_op, {cnn.inputs_p['default']: test_fnames})

i = 0
while True:
    try:
        i += 1
        if i % 5 == 0:
            print(i, end=' ')
        # pull the images out before augmentation occurs; get the labels too or else we'll lose them
        imgs, labels = cnn.sess.run(['IteratorGetNext:0', 'IteratorGetNext:1'])

        # do our own augmentation
        cropped_imgs = sess.run(crop_op, {img_p: imgs})

        # feed the images back in just after augmentation
        preds = cnn.sess.run(cnn.predict, {'cond/Merge:0': cropped_imgs})['default']

        # make sure to get the ordering right so you have the proper 10 images together; average their probabilities
        n_imgs = len(preds) // 10
        preds = np.concatenate([preds[i * n_imgs:(i + 1) * n_imgs].reshape(n_imgs, 100, 1) for i in range(10)], axis=2).mean(axis=2)
        all_preds.append(preds)
        all_labels.append(labels)
    except tf.errors.OutOfRangeError:
        pass

5 10 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/afs/csail.mit.edu/u/n/nhunt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-721cfcfc52de>", line 18, in <module>
    preds = cnn.sess.run(cnn.predict, {'cond/Merge:0': cropped_imgs})['default']
  File "/afs/csail.mit.edu/u/n/nhunt/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 889, in run
    run_metadata_ptr)
  File "/afs/csail.mit.edu/u/n/nhunt/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1120, in _run
    feed_dict_tensor, options, run_metadata)
  File "/afs/csail.mit.edu/u/n/nhunt/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1317, in _do_run
    options, run_metadata)
  File "/afs/csail.mit.edu/u/n/nhunt/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1323, in _do_call
    ret

KeyboardInterrupt: 

In [26]:
img_height = 128
img_width = 128

target_height = 100
target_width = 100

# y0, x0, height, width
upper_left = [0, 0, target_height, target_width]
upper_right = [0, img_width - target_width, target_height, target_width]
lower_left = [img_height - target_height, 0, target_height, target_width]
lower_right = [img_height - target_height, img_width - target_width, target_height, target_width]
center = [(img_height - target_height) // 2, (img_width - target_width) // 2, target_height, target_width]
boxes = [upper_left, upper_right, lower_left, lower_right, center]

img_p = tf.placeholder(tf.uint8, (None, 128, 128, 3))
img_ = tf.image.convert_image_dtype(img_p, tf.float32)
crop_ops = [tf.image.crop_to_bounding_box(img_, *box) for box in boxes]
crop_ops.extend([tf.map_fn(tf.image.flip_left_right, crop) for crop in crop_ops])
crop_op = tf.concat(crop_ops, axis=0)

sess = tf.Session()

In [29]:
p = []
for fname in fnames:
    try:
        cnn = CNN2(models_dir='models/', model_name=fname)
        preds, labels = cnn._batch([cnn.predict['default'], cnn.labels_p['default']], inputs={'default': test_fnames}, dataset=True)
        labels = np.concatenate(labels)
        preds = np.concatenate(preds)

        idx = np.argsort(labels)
        preds = preds[idx]
        p.append(preds)
#         preds, labels = cnn._batch([cnn.predict['default'], cnn.labels_p['default']], inputs={'default': dev_fnames}, dataset=True)
#         labels = np.concatenate(labels)
#         preds = pd.DataFrame(np.concatenate(preds))
#         print(f"{fname}, {acc_at_k(1, preds, labels)}, {acc_at_k(5, preds, labels)}")
    except:
        print(f"Error with {fname}")

Loading graph from: models//inception_aug_LR_contrast_.75_hue_.15_rot_4_l2_0.0001_p_0.8_na_2_nb_2_nc_2_bn_before_True_lr_0.001_e700_bs64/model.ckpt.meta.
Loading graph from: models//inception_aug_LR_contrast_.75_hue_.15_rot_4_l2_0.0001_p_0.8_na_3_nb_5_nc_3_bn_before_True_lr_0.001_e700_bs64/model.ckpt.meta.
